<a href="https://colab.research.google.com/github/ElenaBianchini/ColoringGrayscaleImages/blob/main/ProgettoLabIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Progetto di Laboratorio di Intelligenza Artificiale e Grafica Interattiva**

# Import

In [1]:
%matplotlib inline

In [2]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision 
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from skimage import color

# Impostazione dei parametri

In [3]:
num_epochs = 50
batch_size = 256
learning_rate = 1e-3
use_gpu = torch.cuda.is_available()

# Dataset

In [4]:
# converto ogni immagine in un tensore pytorch contenente un'immagine LAB
def import_image(img):
    return torch.ToTensor(np.transpose(color.rgb2lab(np.array(img)), (2, 0, 1))) # cambiamo l'ordine dei valori perché in skimage il canale è il terzo valore
    
img_transform = transforms.Compose([
    transforms.Resize((128,128)),
    #transforms.Normalize(mean, std), --> calcolare media e deviazione std
    transforms.Lambda(import_image)
])

# imposto small=True perché così so che le immagini sono 256x256
train_dataset = torchvision.datasets.Places365('./data/train', split='train-standard', small = True, transform=img_transform, download=True)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.Places365('./data/test', split='val',  small = True, transform=img_transform, download=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/67498496 [00:00<?, ?it/s]

Extracting ./data/train/filelist_places365-standard.tar to ./data/train


  0%|          | 0/26103685120 [00:00<?, ?it/s]

Extracting ./data/train/train_256_places365standard.tar to ./data/train


OSError: ignored

# Regressione

## Modello

In [ ]:
class ColorizationRNet(nn.Module):
  def __init__(self):
    super(ColorizationRNet, self).__init__()

    # Importo ResNet che userò per estrarre le features dalle immagini
    resnet = torchvision.models.resnet18()
    # Cambio il primo livello di convoluzione di ResNet per accetta input con un solo canale
    resnet.conv1.weight = nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1)) 
    # Estraggo le feature dalle immagini
    self.midlevel_resnet = nn.Sequential(*list(resnet.children())[0:6])

    # Livelli di deconvoluzione:
    self.deconv = nn.Sequential(
        nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Upsample(scale_factor=2),
        nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Upsample(scale_factor=2),
        nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 2, kernel_size=3, stride=1, padding=1),
        nn.Upsample(scale_factor=2)
    )

  def forward(self, x):
    x = self.midlevel_resnet(x)
    output = self.deconv(x)
    return output

In [ ]:
reg_net = ColorizationRNet()

## Funzione di costo e di ottimizzazione

## Train

## Grafico della curva di apprendimento

## Valutazione sul Test Set

## Risultato su un'immagine del Test Set

# Classificazione 

## Modello

In [ ]:
class ColorizationCNet(nn.Module):
  def __init__(self):
    super(ColorizationCNet, self).__init__()

    self.network = nn.Sequential(
        nn.Conv2d(1,64,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(64,64,kernel_size=3, stride=2, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(64),

        nn.Conv2d(64,128,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(128,128,kernel_size=3, stride=2, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(128),

        nn.Conv2d(128,256,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(256,256,kernel_size=3, stride=2, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(256),

        nn.Conv2d(256,512,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512,512,kernel_size=3, stride=2, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(512),

        nn.Conv2d(512,512,kernel_size=3, stride=1, padding=2, dilatation=2),
        nn.ReLU(),
        nn.Conv2d(512,512,kernel_size=3, stride=1, padding=2, dilatation=2),
        nn.ReLU(),
        nn.BatchNorm2d(512),

        nn.Conv2d(512,512,kernel_size=3, stride=1, padding=2, dilatation=2),
        nn.ReLU(),
        nn.Conv2d(512,512,kernel_size=3, stride=1, padding=2, dilatation=2),
        nn.ReLU(),
        nn.BatchNorm2d(512),

        nn.Conv2d(512,512,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(512,512,kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(512),

        nn.ConvTransposed2d(512, 256, kernel_size=4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(256,256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),

        nn.Conv2d(256, 313, kenel_size=1, stride=1, padding=0),
        nn.Softmax(dim=1),
        nn.Conv2d(313, 2, kernel_size=1, padding=0, dilation=1, stride=1),
        nn.Upsample(scale_factor=4)
    )


  def forward(self, x):
    output = self.network(x)
    return output

In [ ]:
clas_net = ColorizationCNet()

## Funzione di costo e di ottimizzazione

## Train

## Grafico della curva di apprendimento

## Valutazione sul Test Set

## Risultato su un'immagine del Test Set